# Transcribe a recording from the user using Wav2Vec2
Rolando Coto-Solano (Rolando.A.Coto.Solano@dartmouth.edu)<br>
Dartmouth College. Last update: 20250601

The program takes two main inputs:

* `currentSandbox`: The name of the sandbox you are using. The defaults are {sandbox-user and all-wavs}, but you can use whichever you specified during the installation.<br>
* `installationFolder`: The folder where the ASR sandboxes are contained. The default value is `202506-ood-asr`, but you should use the one you specified during the installation.<br>

The program takes another input:

* `modelCheckpointToUse`: The name of the folder that has the checkpoint for the transcription model. The default is `checkpoint-1200`, but you should check which one you saved by going to the `wav2vec2-model` folder.<br>

This code loads a language model trained using [XLSR-Wav2Vec2](https://huggingface.co/docs/transformers/model_doc/xlsr_wav2vec2) and uses it to transcribe recordings. The code is based on [Fine-tuning XLS-R for Multi-Lingual ASR with 🤗 Transformers](https://huggingface.co/blog/fine-tune-xlsr-wav2vec2) by [Patrick von Platen](https://huggingface.co/patrickvonplaten).

This file is useful to make a "live demo" of the transcription models.

## (1) Script preparation

In [ ]:
#=================================================
# If the computer tells you to "restart session",
# please restart it and run this box again.
#=================================================

!pip install numpy==1.25.0

In [ ]:
# Environmental variables
currentSandbox = "sandbox-user"             # Please type sandbox-user or all-wavs
installationFolder = "202506-ood-asr"

modelCheckpointToUse = "checkpoint-1200"      # You can type "checkpoint-1200" to use the model you trained

# Use GPU for processing? If you select "no", then the system will use the slower CPU processing
useGPU = "yes"

In [ ]:
# Mount the Google Drive onto the virtual computer

from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

## (2) Model installation

In [ ]:
#=============================================================
# Installs software for ASR model
# Takes about 3~4 minutes
# You only need to run this once per session
#=============================================================

#%%capture
!pip install datasets
!pip install transformers
!pip install librosa
!pip install torch==1.10.0+cu113 torchvision==0.11.1+cu113 torchaudio==0.10.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html  > /dev/null

from transformers import Wav2Vec2ForCTC
from transformers import Wav2Vec2Processor

import glob
import torch
import torchaudio
import pandas as pd
from datasets import Dataset
from transformers import Wav2Vec2FeatureExtractor

#=============================================================
# Determine type of processing
#=============================================================

typeProcessor = "cuda"
if (useGPU == "no"): typeProcessor = "cpu"

#=============================================================
# Downloads ASR model for CIM
#=============================================================

!mkdir /content/wav2vec2-model
!cp /content/drive/MyDrive/{installationFolder}/{currentSandbox}/wav2vec2-model/*.* /content/wav2vec2-model
!mkdir /content/wav2vec2-model/checkpoint
!cp /content/drive/MyDrive/{installationFolder}/{currentSandbox}/wav2vec2-model/{modelCheckpointToUse}/*.* /content/wav2vec2-model/checkpoint

pathCheckpoint = "wav2vec2-model/checkpoint"
model = Wav2Vec2ForCTC.from_pretrained(pathCheckpoint).to(typeProcessor)
processor = Wav2Vec2Processor.from_pretrained("wav2vec2-model")

## (3) Audio Decoding

In [ ]:
# Minimum duration of segments that the computer should transcribe
minDurationOfFile = 100 #ms


#=============================================================
# Create dummy CSV file
#=============================================================

output = "path,sentence\nsample-recording.wav,kia orana"
f = open("sample-recording.csv", "w")
f.write(output)
f.close()


#=============================================================
# Convert model orthography to human orthography
#=============================================================

def convertToHumanOrthography(inputString):

  outputString = inputString

  # Replace the output transcription with orthographic output
  orthOrigin = ['ax', 'ex', 'ix', 'ox', 'ux', 'q']
  orthTarget = ['ā', 'ē', 'ī', 'ō', 'ū', 'ꞌ']
  for i in range(0,len(orthOrigin)): outputString = outputString.replace(orthOrigin[i], orthTarget[i])

  return outputString


#=============================================================
# Function to run inference of new files
#=============================================================

def runInference ():

  # Load models
  #pathCheckpoint = "cim-checkpoint-lrec/checkpoint"
  #model = Wav2Vec2ForCTC.from_pretrained(pathCheckpoint).to("cuda")
  #processor = Wav2Vec2Processor.from_pretrained("cim-checkpoint-lrec")

  # Convert audio file to array
  def speech_file_to_array_fn(batch):
      speech_array, sampling_rate = torchaudio.load(batch["path"])
      batch["speech"] = speech_array[0].numpy()
      batch["sampling_rate"] = sampling_rate
      batch["target_text"] = batch["sentence"]
      return batch

  # Prepare batch processing of files
  def prepare_dataset(batch):
      # check that all files have the correct sampling rate
      assert (
          len(set(batch["sampling_rate"])) == 1
      ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

      batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values

      with processor.as_target_processor():
          batch["labels"] = processor(batch["target_text"]).input_ids
      return batch

  # Load CSV file with audio files to be transcribed
  dataTest = pd.read_csv("sample-recording.csv")
  common_voice_test = Dataset.from_pandas(dataTest)

  # Extract features from audio files
  feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)
  common_voice_test = common_voice_test.map(speech_file_to_array_fn, remove_columns=common_voice_test.column_names)
  common_voice_test = common_voice_test.map(prepare_dataset, remove_columns=common_voice_test.column_names, batch_size=8, num_proc=4, batched=True)

  # Process audio files
  input_dict = processor(common_voice_test[0]["input_values"], return_tensors="pt", padding=True)
  logits = model(input_dict.input_values.to(typeProcessor)).logits
  pred_ids = torch.argmax(logits, dim=-1)[0]

  # Decode audio files
  predictedText = convertToHumanOrthography(processor.decode(pred_ids))

  return predictedText

## (4) Inference: Transcribing new files

In [ ]:
#=============================================================
# Code to record audio from Colab Notebook
# Takes about 30 seconds
#=============================================================

!apt-get install libsox-fmt-all libsox-dev sox > /dev/null
!python -m pip install git+https://github.com/facebookresearch/WavAugment.git > /dev/null
!pip install ffmpeg-python > /dev/null

import torchaudio

# code from https://ricardodeazambuja.com/deep_learning/2019/03/09/audio_and_video_google_colab/
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
import io
import ffmpeg
import tempfile
import pathlib


AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data);
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});

</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])

  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)

  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  with tempfile.TemporaryDirectory() as tmpdirname:
    path = pathlib.Path(tmpdirname) / 'tmp.wav'
    with open(path, 'wb') as f:
       f.write(riff)
    with open('temp.wav', 'wb') as f:
       f.write(riff)

    x, sr = torchaudio.load(path)

  return x, sr

In [ ]:
# GET READY!
# When you run the next box, the computer will say that it's listening to you.
# I recommend that you say something simple. For example:
#
# Kia orana!
#
# It means "Hello" or "How are you?"
# After you said it, click on "Press to stop". Once the "play" button (on the left) stops rotating, that's
# when your recording will be ready and you can continue to the next step. (The button will keep saying
# "saving the recording... pls wait!". Don't pay attention to that. Pay attention to the play button. If
# the rotating line has stopped, then you're good).
#
# The system will make many mistakes because it's only trained on 16 minutes on audio.
# As the system learns from more data, the transcriptions will get better.

In [ ]:
#==========================================================================
# This will open a button to record audio.
# It will save the file as temp.wav
# Once you see a play button, this means that the file is saved correctly.
#==========================================================================

%rm /content/temp.wav
%cd /content/
x, sr = get_audio()

In [ ]:
#==========================================================================
# Convert the audio file to mono (one channel)
# and downgrade its quality to 16KHz
#==========================================================================

!ffmpeg -y -i temp.wav -ac 1 -ar 16000 temp-sample-recording.wav
!rm temp.wav
!mv temp-sample-recording.wav sample-recording.wav

In [ ]:
#==========================================================================
# Transcribe recording
#==========================================================================

predictedText = runInference()

In [ ]:
#==========================================================================
# Print the transcription of the recording
#==========================================================================

print("Prediction:")
print(predictedText)

In [ ]:
#==========================================================================
# Play recording
#==========================================================================

import IPython
IPython.display.Audio('sample-recording.wav') # This is required on Google Colab due to compatibility issues